# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 10:10AM,238633,15,PNC363319A,"Person & Covey, Inc.",Released
1,Jun 2 2022 10:10AM,238633,15,PNC363320A,"Person & Covey, Inc.",Released
2,Jun 2 2022 10:10AM,238633,15,PNC363321A,"Person & Covey, Inc.",Released
3,Jun 2 2022 10:10AM,238633,15,PNC363322A,"Person & Covey, Inc.",Released
4,Jun 2 2022 10:10AM,238633,15,PNC363324A,"Person & Covey, Inc.",Released
5,Jun 2 2022 10:10AM,238633,15,PNC363325A,"Person & Covey, Inc.",Released
6,Jun 2 2022 10:10AM,238633,15,PNC363326A,"Person & Covey, Inc.",Released
7,Jun 2 2022 10:10AM,238633,15,PNC363327A,"Person & Covey, Inc.",Released
8,Jun 2 2022 10:10AM,238633,15,PNC363337A,"Person & Covey, Inc.",Released
9,Jun 2 2022 10:10AM,238633,15,PNC363302A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,238630,Released,13
47,238631,Released,14
48,238633,Released,17
49,238634,Released,1
50,238639,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238630,NaN,13.0
238631,NaN,14.0
238633,NaN,17.0
238634,NaN,1.0
238639,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238524,0.0,1.0
238526,0.0,1.0
238529,1.0,59.0
238544,10.0,11.0
238551,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238524,0,1
238526,0,1
238529,1,59
238544,10,11
238551,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238524,0,1
1,238526,0,1
2,238529,1,59
3,238544,10,11
4,238551,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238524,,1
1,238526,,1
2,238529,1,59
3,238544,10,11
4,238551,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 10:10AM,238633,15,"Person & Covey, Inc."
17,Jun 2 2022 10:07AM,238639,16,American International Chemical
18,Jun 2 2022 10:05AM,238631,15,"Person & Covey, Inc."
32,Jun 2 2022 10:04AM,238564,20,"ACI Healthcare USA, Inc."
39,Jun 2 2022 10:03AM,238628,10,ISDIN Corporation
80,Jun 2 2022 10:03AM,238626,10,ISDIN Corporation
121,Jun 2 2022 10:02AM,238630,15,"Person & Covey, Inc."
134,Jun 2 2022 10:00AM,238634,12,Proven Skincare
135,Jun 2 2022 9:59AM,238625,15,"Virtus Pharmaceuticals, LLC"
136,Jun 2 2022 9:55AM,238598,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 2 2022 10:10AM,238633,15,"Person & Covey, Inc.",,17
1,Jun 2 2022 10:07AM,238639,16,American International Chemical,,1
2,Jun 2 2022 10:05AM,238631,15,"Person & Covey, Inc.",,14
3,Jun 2 2022 10:04AM,238564,20,"ACI Healthcare USA, Inc.",,7
4,Jun 2 2022 10:03AM,238628,10,ISDIN Corporation,,41
5,Jun 2 2022 10:03AM,238626,10,ISDIN Corporation,,41
6,Jun 2 2022 10:02AM,238630,15,"Person & Covey, Inc.",,13
7,Jun 2 2022 10:00AM,238634,12,Proven Skincare,,1
8,Jun 2 2022 9:59AM,238625,15,"Virtus Pharmaceuticals, LLC",,1
9,Jun 2 2022 9:55AM,238598,15,"Mizner Bioscience, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 10:10AM,238633,15,"Person & Covey, Inc.",17,
1,Jun 2 2022 10:07AM,238639,16,American International Chemical,1,
2,Jun 2 2022 10:05AM,238631,15,"Person & Covey, Inc.",14,
3,Jun 2 2022 10:04AM,238564,20,"ACI Healthcare USA, Inc.",7,
4,Jun 2 2022 10:03AM,238628,10,ISDIN Corporation,41,
5,Jun 2 2022 10:03AM,238626,10,ISDIN Corporation,41,
6,Jun 2 2022 10:02AM,238630,15,"Person & Covey, Inc.",13,
7,Jun 2 2022 10:00AM,238634,12,Proven Skincare,1,
8,Jun 2 2022 9:59AM,238625,15,"Virtus Pharmaceuticals, LLC",1,
9,Jun 2 2022 9:55AM,238598,15,"Mizner Bioscience, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 10:10AM,238633,15,"Person & Covey, Inc.",17,
1,Jun 2 2022 10:07AM,238639,16,American International Chemical,1,
2,Jun 2 2022 10:05AM,238631,15,"Person & Covey, Inc.",14,
3,Jun 2 2022 10:04AM,238564,20,"ACI Healthcare USA, Inc.",7,
4,Jun 2 2022 10:03AM,238628,10,ISDIN Corporation,41,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 10:10AM,238633,15,"Person & Covey, Inc.",17.0,NaN
1,Jun 2 2022 10:07AM,238639,16,American International Chemical,1.0,NaN
2,Jun 2 2022 10:05AM,238631,15,"Person & Covey, Inc.",14.0,NaN
3,Jun 2 2022 10:04AM,238564,20,"ACI Healthcare USA, Inc.",7.0,NaN
4,Jun 2 2022 10:03AM,238628,10,ISDIN Corporation,41.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 10:10AM,238633,15,"Person & Covey, Inc.",17.0,0.0
1,Jun 2 2022 10:07AM,238639,16,American International Chemical,1.0,0.0
2,Jun 2 2022 10:05AM,238631,15,"Person & Covey, Inc.",14.0,0.0
3,Jun 2 2022 10:04AM,238564,20,"ACI Healthcare USA, Inc.",7.0,0.0
4,Jun 2 2022 10:03AM,238628,10,ISDIN Corporation,41.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2147507,121.0,7.0
12,1431504,71.0,1.0
15,1908929,92.0,0.0
16,954242,3.0,1.0
19,954344,30.0,42.0
20,3101650,42.0,17.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2147507,121.0,7.0
1,12,1431504,71.0,1.0
2,15,1908929,92.0,0.0
3,16,954242,3.0,1.0
4,19,954344,30.0,42.0
5,20,3101650,42.0,17.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,121.0,7.0
1,12,71.0,1.0
2,15,92.0,0.0
3,16,3.0,1.0
4,19,30.0,42.0
5,20,42.0,17.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,121.0
1,12,Released,71.0
2,15,Released,92.0
3,16,Released,3.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,7.0,1.0,0.0,1.0,42.0,17.0
Released,121.0,71.0,92.0,3.0,30.0,42.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,7.0,1.0,0.0,1.0,42.0,17.0
1,Released,121.0,71.0,92.0,3.0,30.0,42.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,7.0,1.0,0.0,1.0,42.0,17.0
1,Released,121.0,71.0,92.0,3.0,30.0,42.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()